# Quantifying Dynamic Risk Factors for Vehicular Crashes

## Motivation
Briefly state the nature of your project and why you chose it. What specific question or goal did you try to address?

- show risks of injury and deaths based on four common driver errors
- Calculate probabilites of accident while driving poorly.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
from altair import datum

## Data Sources
- where the datasets or API resources are located,
- what formats they returned/used,
- what were the important variables contained in them,
- how many records you used or retrieved (if using an API), and
- what time periods they covered (if there is a time element)

In [3]:
# Import data ##########################################################
df_vehicle = pd.read_csv("data/crss_2021_csv/vehicle.csv")
df_person = pd.read_csv("data/crss_2021_csv/person.csv")
df_accident = pd.read_csv("data/crss_2021_csv/accident.csv")

# Daniel's Topics: Fast driving, unbuckled driving
# relate to accident type, injury, and damage
# indicators: SPEEDREL, VSPD_LIM, TRAV_SP, CASENUM
# TODO look at accident dataframe and generate basic statistics

## Cleaning and Manipulation

- How specifically did you need to manipulate the data?
- How did you handle missing, incomplete, or incorrect data?
- How did you perform conversion or processing steps?
- What variables and steps did you use to join the two data resources to perform your data analysis?
- What challenges did you encounter and how did you solve them?

In [4]:
# Data cleaning and manipulation #######################################

# Replace non-applicable codes with NaN
# for plotting purposes
replace_vals_trav_speed = {997: np.NaN, 998: np.NaN, 999: np.NaN}
replace_vals_speed_lim = {98: np.NaN, 99: np.NaN}
replace_vals_age = {998: np.NaN, 999: np.NaN}

df_vehicle["TRAV_SP_1"] = df_vehicle["TRAV_SP"].replace(replace_vals_trav_speed)
df_vehicle["VSPD_LIM_1"] = df_vehicle["VSPD_LIM"].replace(replace_vals_speed_lim)
df_person["AGE_1"] = df_person["AGE"].replace(replace_vals_age)

# Merge vehicle and person df by CASENUM
df_merged = df_vehicle.merge(df_person, how="outer", on="CASENUM")
# Merge accident df by CASENUM
df_merged = df_merged.merge(df_accident, how="outer", on="CASENUM")

# Bin travel_speed
bins = np.arange(0, 105, 5)
bin_labels = np.arange(0, 105, 5)[1:]
df_merged["TRAV_SP_1_bins"] = pd.cut(
    df_merged["TRAV_SP_1"], bins=bins, labels=bin_labels
)

# TODO filter based on speed related crashes? df_vehicle['SPEEDREL']

In [5]:
# TODO addressed unknown values with imputed
# df_merged[['INJ_SEV','INJSEV_IM']].head(50)

Define imputed values. 
Impact of using imputed values p.141
Percentages of Unknown and Not Reported Values p.14

Used Indicators:

    1. TRAV_SP
        - TRAV_SP_1
        - TRAV_SP_1_bins

    2. VSPD_LIM
        - VSPD_LIM_1
        
    3. INJ_SEVNAME
        - (INJSEV_IM)

    4. EJECTIONNAME
        - (EJECTTION, EJECT_IM)
    
    5. REST_USENAME

In [6]:
# TODO Generate basic statistics for accident data
# time of day, day of week, holidays
# visibility
# weather

Probability based on relative incidences 
- Exmaple: 65 mph speed limiit 5x greater deaths than 40 mph speed limit


## Visualizations

All visualizations referenced and explained in the text. Visualizations are complete, including appropriate title, axis labels, etc. Visualizations are annotated appropriately (note: not all visualizations need annotations).

In [25]:
# travel speed vs speed limit ########################################################################
def point_plot(df):

    # Reduce size of dataframe by only including relevant columns
    columns=['TRAV_SP_1_bins', 'VSPD_LIM_1']
    df = df[columns]

    bars = alt.Chart(df).mark_point().encode(
    y=alt.Y('TRAV_SP_1_bins:Q',  axis=alt.Axis(values=list(range(0, 100, 5))), title='Driver Speed'),
    x=alt.X('VSPD_LIM_1:Q', axis=alt.Axis(values=list(range(0, 100, 5))), title='Speed Limit'),
    size=alt.Size('TRAV_SP_1_bins', aggregate='count')
    ).properties(
        width=300,
        height=300
    ).configure_axis(
        labelFontSize=14,
        titleFontSize=14
    )
    return bars
        
point_plot(df_merged)

alt.Chart(...)

In [24]:
def heatmap_plot(df):
    
    # Reduce size of dataframe by only including relevant columns
    columns=['TRAV_SP_1', 'VSPD_LIM_1', 'TRAV_SP_1_bins']
    df = df[columns]

    heatmap = alt.Chart(df).mark_rect().encode(
        alt.Y('TRAV_SP_1:Q', title='Driver Speed').bin(maxbins=20),
        alt.X('VSPD_LIM_1:Q', title='Speed Limit').bin(maxbins=20),
        alt.Color('count():Q').scale(scheme='greenblue')
    ).transform_filter(
        (datum.TRAV_SP_1_bins <= 80)# & (datum.sex == 1)
    ).properties(
        width=300,
        height=300
    ).configure_axis(
        labelFontSize=14,
        titleFontSize=14
    )
    return heatmap

heatmap_plot(df_merged)

alt.Chart(...)

In [23]:
# INJSEV_IM
# TODO use pandas cut,
# TODO pandas groupby, plot travel speed vs count(INJ_SEV),

# 0 No Apparent Injury (O)
# 1 Possible Injury (C)
# 2 Suspected Minor Injury (B)
# 3 Suspected Serious Injury (A)
# 4 Fatal Injury (K)
# 5 Injured, Severity Unknown (U)
# 6 Died Prior to Crash
# 9 Unknown/Not Reported

def point_plot_1(df):

    # Reduce size of dataframe by only including relevant columns
    columns=['TRAV_SP_1_bins', 'INJ_SEVNAME']
    df = df[columns]
    bars = alt.Chart(df).mark_point().encode(
        x=alt.X('TRAV_SP_1_bins:Q',  axis=alt.Axis(values=list(range(0, 100, 5))), title='Driver Speed'),
        y=alt.Y('INJ_SEVNAME:N', title='Injury Severity'),
        size=alt.Size('TRAV_SP_1_bins', aggregate='count')
    ).properties(
        width=500,
        height=500
    ).configure_axis(
        labelFontSize=14,
        titleFontSize=14
    )
    return bars

point_plot_1(df_merged)

alt.Chart(...)

In [10]:
df_merged['INJ_SEVNAME'].unique()

array(['No Apparent Injury (O)', 'Suspected Minor Injury (B)',
       'Suspected Serious Injury (A)', 'Fatal Injury (K)',
       'Unknown/Not Reported', 'Possible Injury (C)', nan], dtype=object)

In [21]:
def injury_speed_dist_plot(df):
    
    # Reduce size of dataframe by only including relevant columns
    columns=['VSPD_LIM_1', 'INJ_SEVNAME', 'TRAV_SP_1']
    df = df[columns]

    speed_limit = alt.Chart(df).mark_area().encode(
        x=alt.X('VSPD_LIM_1:Q', title='Speed Limit (mph)'),
        y=alt.Y('count(VSPD_LIM_1):Q', title="",scale=alt.Scale(domain=[0,200])).scale(type="log"),
        color=alt.Color('INJ_SEVNAME:N', legend=None, title=''),
        row=alt.Row('INJ_SEVNAME:N',header=alt.Header(labelAngle=0, labelAlign='left'),title='COUNT') #.sort(['MSFT', 'AAPL', 'IBM', 'AMZN'])
    ).transform_filter(
        (datum.INJ_SEVNAME == 'No Apparent Injury (O)') | (datum.INJ_SEVNAME == 'Suspected Minor Injury (B)') |
        (datum.INJ_SEVNAME == 'Suspected Serious Injury (A)') | (datum.INJ_SEVNAME == 'Fatal Injury (K)') |
        (datum.INJ_SEVNAME == 'Unknown/Not Reported') | (datum.INJ_SEVNAME == 'Possible Injury (C)')

    ).properties(height=60, width=400
    )

    driver_speed = alt.Chart(df).mark_area().encode(
        x=alt.X('TRAV_SP_1:Q', title='Driver Speed (mph)'),
        y=alt.Y('count(TRAV_SP_1):Q', title="",scale=alt.Scale(domain=[0,200])).scale(type="log"),
        color=alt.Color('INJ_SEVNAME:N', legend=None, title=''),
        row=alt.Row('INJ_SEVNAME:N',header=None,title='') #.sort(['MSFT', 'AAPL', 'IBM', 'AMZN'])
    ).transform_filter(
        (datum.INJ_SEVNAME == 'No Apparent Injury (O)') | (datum.INJ_SEVNAME == 'Suspected Minor Injury (B)') |
        (datum.INJ_SEVNAME == 'Suspected Serious Injury (A)') | (datum.INJ_SEVNAME == 'Fatal Injury (K)') |
        (datum.INJ_SEVNAME == 'Unknown/Not Reported') | (datum.INJ_SEVNAME == 'Possible Injury (C)')

    ).properties(height=60, width=400
    )

    combined = (speed_limit | driver_speed).configure_axis(
        labelFontSize=12,
        titleFontSize=12
    )
    return combined

injury_speed_dist_plot(df_merged)

alt.HConcatChart(...)

In [20]:
def bar_driver_speed_dist(df):

    # Reduce size of dataframe by only including relevant columns
    columns=['TRAV_SP_1']
    df = df[columns]

    # Normalize percent of accidents based on driver speed
    bar = alt.Chart(df, title='Percent of Accidents based on Driver Speed').transform_aggregate(
        count='count()',
        groupby=['TRAV_SP_1']
    ).transform_joinaggregate(
        total='sum(count)'
    ).transform_calculate(
        frac='datum.count / datum.total'
    ).mark_bar(size=10).encode(
        x=alt.X('TRAV_SP_1:Q'),
        y=alt.Y('frac:Q', title='Percent of Accidents',axis=alt.Axis(format='%')),
    ).properties(
        width=1000,
        height=500
    ).configure_axis(
        labelFontSize=14,
        titleFontSize=14
    ).configure_title(fontSize=24)
    return bar

bar_driver_speed_dist(df_merged)

alt.Chart(...)

In [19]:
def bar_speed_lim_dist(df):

    # Reduce size of dataframe by only including relevant columns
    columns=['VSPD_LIM_1']
    df = df[columns]

    # Normalize percent of accidents based on speed limit
    bar = alt.Chart(df, title='Percent of Accidents based on Speed Limit').transform_aggregate(
        count='count()',
        groupby=['VSPD_LIM_1']
    ).transform_joinaggregate(
        total='sum(count)'
    ).transform_calculate(
        frac='datum.count / datum.total'
    ).mark_bar(size=10).encode(
        x=alt.X('VSPD_LIM_1:Q'),
        y=alt.Y('frac:Q', title='Percent of Accidents',axis=alt.Axis(format='%')),
    ).properties(
        width=1000,
        height=500
    ).configure_axis(
        labelFontSize=14,
        titleFontSize=14
    ).configure_title(fontSize=24)
    return bar

bar_speed_lim_dist(df_merged)

alt.Chart(...)